In [2]:
import pandas as pd

In [7]:
movies = pd.read_csv('C:\\Users\\Dmitry\\Documents\\Учеба\\5 Аналитика Big Data\\5.12 Итоговая работа\\uitem.csv', sep='|', encoding = 'Windows-1251')

In [13]:
movies.columns = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_url', 'unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western']

In [14]:
movies.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_url,unknown,action,adventure,animation,childrens,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
0,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
ratings = pd.read_csv('C:\\Users\\Dmitry\\Documents\\Учеба\\5 Аналитика Big Data\\5.12 Итоговая работа\\u.data.csv', sep ='\t')

In [11]:
ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [12]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


#### 2. Пользователь с наибольшим числом оценок

In [21]:
ratings.groupby('user_id').count().sort_values('rating', ascending = False).reset_index().head()

,user_id,movie_id,rating,timestamp
0,405,737,737,737
1,655,685,685,685
2,13,636,636,636
3,450,540,540,540
4,276,518,518,518


#### 3. df ratings только с фильмами, которые оценил самый активный пользователь

In [16]:
ratings405 = ratings[ratings['user_id'] == 405]
ratings405.head()

,user_id,movie_id,rating,timestamp
12275,405,56,4,885544911
12382,405,592,1,885548670
12429,405,1582,1,885548670
12448,405,171,1,885549544
12459,405,580,1,885547447


####  4. Добавление столбцов

In [29]:
# общее количество оценок фильмов
num_rating = ratings.groupby('movie_id').count().reset_index()[['movie_id', 'rating']]
num_rating = num_rating.rename(columns = {'rating' : 'num_of_ratings'})
num_rating.head()

,movie_id,num_of_ratings
0,1,452
1,2,131
2,3,90
3,4,209
4,5,86


In [30]:
# суммарная оценка от всех пользователей
sum_rating = ratings.groupby('movie_id').sum().reset_index()[['movie_id', 'rating']]
sum_rating = sum_rating.rename(columns = {'rating' : 'sum_of_ratings'})
sum_rating.head()

,movie_id,sum_of_ratings
0,1,1753
1,2,420
2,3,273
3,4,742
4,5,284


In [77]:
ratings405_joined = ratings405.merge(num_rating, how='left', on='movie_id').merge(sum_rating, how='left', on='movie_id').merge(movies, how='left', on='movie_id')[['user_id', 'rating', 'movie_id', 'num_of_ratings', 'sum_of_ratings', 'unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western']]
ratings405_joined.head()

,user_id,rating,movie_id,num_of_ratings,sum_of_ratings,unknown,action,adventure,animation,childrens,...,fantasy,film_noir,horror,musical,mystery,romance,sci_fi,thriller,war,western
0,405,4,56,394,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,1,592,9,30,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,405,1,1582,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,405,1,171,65,252,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,405,1,580,32,108,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


#### 5. Сформируйте X_train, X_test, y_train, y_test

In [78]:
from sklearn.model_selection import train_test_split

In [80]:
x_train, x_test, y_train, y_test = train_test_split(ratings405_joined[['movie_id', 'num_of_ratings', 'sum_of_ratings', 'unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'war', 'western']], ratings405_joined['rating'], test_size = 0.3) 

#### 6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [81]:
from sklearn.linear_model import LinearRegression

In [86]:
model = LinearRegression()

In [87]:
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#### 7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [90]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [92]:
# в идеале должно быть 0
mean_squared_error(y_test, model.predict(x_test))

1.4974999006007026

In [94]:
# в идеале должно быть 1
r2_score(y_test, model.predict(x_test))

0.24021181538044578

In [96]:
# в идеале должно быть 0
mean_absolute_error(y_test, model.predict(x_test))

0.9128032810696576

Судя по оценкам качества, модель получилась плохая